<a href="https://colab.research.google.com/github/SaumyajitDas001/COMSYS_Hackathon/blob/main/TaskB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! gdown --id 1gRS3YMBCWKy_HkpvtdeoLHsc9RApmB3U

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1gRS3YMBCWKy_HkpvtdeoLHsc9RApmB3U
From (redirected): https://drive.google.com/uc?id=1gRS3YMBCWKy_HkpvtdeoLHsc9RApmB3U&confirm=t&uuid=73fac43c-8fcc-41ea-8c28-a002063c9791
To: /content/TaskB.zip
100% 2.22G/2.22G [00:38<00:00, 57.1MB/s]


In [ ]:
zip_path = "/content/TaskB.zip"

import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content")

In [ ]:
import os
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import accuracy_score

In [ ]:
!ls /content/Task_B/val
!ls /content/Task_B/val/Duane_Lee_Chapman/distortion

009_frontal	       Duane_Lee_Chapman	Mario_Lobo_Zagallo
017_frontal	       Edina_Batar		Mark_Broxmeyer
018_frontal	       Elaine_Chao		Mark_Warner
023_frontal	       Elizabeth_Hill		Martin_Brodeur
024_frontal	       Elizabeth_Taylor		Martin_Luther_King_III
033_frontal	       Eliza_Manningham-Buller	Mary_Jo_Myers
034_frontal	       Emanuel_Ginobili		Matt_Braker
037_frontal	       Enrique_Bolanos		Melana_Scantlin
041_frontal	       Eric_Hinske		Melissa_Manchester
046_frontal	       Eva_Herzigova		Mia_Mottley
049_frontal	       Evie_Lazarou		Michael_Kirby
050_frontal	       Fernando_Valenzuela	Michele_Placido
052_frontal	       Francisco_Flores		Mickey_Gilley
058_frontal	       Fran_Drescher		Miguel_Jimenez
061_frontal	       Franz_Gsell		Mike_Maroth
069_frontal	       Gabi_Zimmer		Mike_Sherman
070_frontal	       Garry_Kasparov		Milan_Kucan
078_frontal	       Gary_Bettman		Millicent_Martin
079_frontal	       Gene_Autry		Mitzi_Gaynor
080_frontal	       George_Blaney		Miyako_Miyazaki
083_fr

In [ ]:
class CleanDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        self.class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {name: idx for idx, name in enumerate(self.class_names)}

        for class_name in self.class_names:
            class_path = os.path.join(root_dir, class_name)
            for img in os.listdir(class_path):
                if img.lower().endswith(('.jpg', '.png')) and "distorted" not in img.lower():
                    self.image_paths.append(os.path.join(class_path, img))
                    self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

class DistortedDataset(Dataset):
    def __init__(self, root_dir, class_to_idx, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform

        for class_name in os.listdir(root_dir):
            distorted_path = os.path.join(root_dir, class_name, "distortion")

            if not os.path.isdir(distorted_path):
                print(f"⚠️ Skipped: {distorted_path} is not a valid folder.")
                continue

            for img in os.listdir(distorted_path):
                if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                    full_path = os.path.join(distorted_path, img)
                    self.image_paths.append(full_path)
                    self.labels.append(class_to_idx[class_name])

        print(f"Loaded {len(self.image_paths)} distorted images.")
        if len(self.image_paths) == 0:
            print("No images found in any Distorted/ folders!")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label


In [ ]:
import os

train_path = '/content/Task_B/train'
val_path = '/content/Task_B/val'

all_class_names = sorted(list(set(os.listdir(train_path)) | set(os.listdir(val_path))))
class_to_idx = {name: idx for idx, name in enumerate(all_class_names)}

print("Total classes (combined train + val):", len(class_to_idx))
print("class_to_idx:", class_to_idx)

Total classes (combined train + val): 1127
class_to_idx: {'001_frontal': 0, '002_frontal': 1, '003_frontal': 2, '004_frontal': 3, '005_frontal': 4, '007_frontal': 5, '008_frontal': 6, '009_frontal': 7, '010_frontal': 8, '011_frontal': 9, '012_frontal': 10, '013_frontal': 11, '014_frontal': 12, '015_frontal': 13, '016_frontal': 14, '017_frontal': 15, '018_frontal': 16, '019_frontal': 17, '020_frontal': 18, '021_frontal': 19, '022_frontal': 20, '023_frontal': 21, '024_frontal': 22, '025_frontal': 23, '026_frontal': 24, '027_frontal': 25, '028_frontal': 26, '029_frontal': 27, '030_frontal': 28, '031_frontal': 29, '032_frontal': 30, '033_frontal': 31, '034_frontal': 32, '035_frontal': 33, '036_frontal': 34, '037_frontal': 35, '038_frontal': 36, '039_frontal': 37, '041_frontal': 38, '042_frontal': 39, '043_frontal': 40, '044_frontal': 41, '045_frontal': 42, '046_frontal': 43, '047_frontal': 44, '048_frontal': 45, '049_frontal': 46, '050_frontal': 47, '051_frontal': 48, '052_frontal': 49, '0

In [ ]:
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = CleanDataset(train_path, transform=transform)
train_dataset.class_to_idx = class_to_idx

val_dataset = DistortedDataset(val_path, class_to_idx=class_to_idx, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)


✅ Loaded 2954 distorted images.


In [ ]:
class FaceNet(nn.Module):
    def __init__(self, embedding_dim=128):
        super(FaceNet, self).__init__()
        self.base = models.resnet18(pretrained=True)
        self.base.fc = nn.Identity()
        self.embedding = nn.Linear(512, embedding_dim)

    def forward(self, x):
        x = self.base(x)
        x = self.embedding(x)
        return x

class Classifier(nn.Module):
    def __init__(self, embedding_dim, num_classes):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FaceNet().to(device)
classifier = Classifier(128, len(train_dataset.class_names)).to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(list(model.parameters()) + list(classifier.parameters()), lr=1e-4)

for epoch in range(10):
    model.train()
    classifier.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        embeddings = model(images)
        outputs = classifier(embeddings)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


Epoch 1, Loss: 370.5749
Epoch 2, Loss: 292.2340
Epoch 3, Loss: 241.8777
Epoch 4, Loss: 200.3042
Epoch 5, Loss: 158.0059
Epoch 6, Loss: 122.5898
Epoch 7, Loss: 90.0765
Epoch 8, Loss: 62.1411
Epoch 9, Loss: 41.8493
Epoch 10, Loss: 26.9883


In [ ]:
model.eval()
classifier.eval()

preds = []
trues = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        emb = model(images)
        out = classifier(emb)
        pred = torch.argmax(out, dim=1)
        preds.append(pred.item())
        trues.append(labels.item())
print("Number of distorted images evaluated:", len(preds))

acc = accuracy_score(trues, preds)
print(f"Validation Accuracy on Distorted Images: {acc:.2%}")

Number of distorted images evaluated: 2954
Validation Accuracy on Distorted Images: 0.30%


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
if len(preds) > 0:
    accuracy = accuracy_score(trues, preds)
    precision_macro = precision_score(trues, preds, average='macro')
    recall_macro = recall_score(trues, preds, average='macro')
    f1_macro = f1_score(trues, preds, average='macro')

    precision_weighted = precision_score(trues, preds, average='weighted')
    recall_weighted = recall_score(trues, preds, average='weighted')
    f1_weighted = f1_score(trues, preds, average='weighted')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Macro): {precision_macro:.4f}")
    print(f"Recall (Macro): {recall_macro:.4f}")
    print(f"F1 Score (Macro): {f1_macro:.4f}")

    print(f"Precision (Weighted): {precision_weighted:.4f}")
    print(f"Recall (Weighted): {recall_weighted:.4f}")
    print(f"F1 Score (Weighted): {f1_weighted:.4f}")

else:
    print("No predictions were made. Check dataset or model output.")


Accuracy: 0.0030
Precision (Macro): 0.0039
Recall (Macro): 0.0023
F1 Score (Macro): 0.0023
Precision (Weighted): 0.0048
Recall (Weighted): 0.0030
F1 Score (Weighted): 0.0029


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_